In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from orbit.models.lgt import LGTMAP, LGTAggregated, LGTFull
from orbit.models.dlt import DLTMAP, DLTAggregated, DLTFull
from orbit.utils.dataset import load_iclaims

from orbit.diagnostics.backtest import grid_search_orbit
from orbit.diagnostics.metrics import smape, mape, wmape

## Load Data

In [2]:
raw_data = load_iclaims()
data = raw_data.copy()

print(data.shape)
data.head(5)

(443, 5)


,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,13.386595,0.168876,-0.328309,0.113033
1,2010-01-10,13.624218,0.168876,-0.204695,0.164326
2,2010-01-17,13.398741,0.185136,-0.302334,0.113033
3,2010-01-24,13.137549,0.152346,-0.204695,0.102451
4,2010-01-31,13.196760,0.083354,-0.252323,0.070016


## LGT Tuning Exampe

In [3]:
lgt = LGTMAP(
    date_col='week',
    response_col='claims',
    seasonality=52,
)

In [4]:
param_grid = {
    'level_sm_input': [0.3, 0.5, 0.8],  
    'seasonality_sm_input': [0.3, 0.5, 0.8],
}

In [5]:
best_params, tuned_df = grid_search_orbit(param_grid, 
                                        model=lgt, 
                                        df=data,
                                        min_train_len=100, incremental_len=100, forecast_len=20, 
                                        metrics=None, criteria=None, verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.3}


 11%|█         | 1/9 [00:00<00:03,  2.23it/s]

tuning metric:0.0066448
tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.5}


 22%|██▏       | 2/9 [00:00<00:02,  2.43it/s]

...

tuning metric:0.0093422
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.5}


 89%|████████▉ | 8/9 [00:03<00:00,  2.11it/s]

tuning metric:0.0093191
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.8}


100%|██████████| 9/9 [00:03<00:00,  2.29it/s]

tuning metric:0.008009


In [6]:
tuned_df.head()

,level_sm_input,seasonality_sm_input,metrics
0,0.3,0.3,0.006645
1,0.3,0.5,0.006039
2,0.3,0.8,0.005651
3,0.5,0.3,0.007694
4,0.5,0.5,0.006919


In [7]:
best_params

[{'level_sm_input': 0.3, 'seasonality_sm_input': 0.8}]

## DLT Tuning Example

In [8]:
dlt = DLTMAP(
    date_col='week',
    response_col='claims',
    regressor_col=['trend.unemploy', 'trend.filling', 'trend.job'],
    seasonality=52,
)

In [9]:
param_grid = {
    'regressor_beta_prior': [[0] * 3, [0.05] * 3, [0.1] * 3],  
    'regressor_sigma_prior': [[0.3] * 3, [0.5] * 3, [1] * 3],
}

In [10]:
best_params, tuned_df = grid_search_orbit(param_grid, 
                                        model=dlt, 
                                        df=data,
                                        min_train_len=100, incremental_len=100, forecast_len=20, 
                                        metrics=None, criteria=None, verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'regressor_beta_prior': [0, 0, 0], 'regressor_sigma_prior': [0.3, 0.3, 0.3]}


 11%|█         | 1/9 [00:00<00:04,  1.63it/s]

tuning metric:0.0066081
tuning hyper-params {'regressor_beta_prior': [0, 0, 0], 'regressor_sigma_prior': [0.5, 0.5, 0.5]}


 22%|██▏       | 2/9 [00:01<00:03,  1.77it/s]

...

tuning metric:0.0064333
tuning hyper-params {'regressor_beta_prior': [0.1, 0.1, 0.1], 'regressor_sigma_prior': [0.5, 0.5, 0.5]}


 89%|████████▉ | 8/9 [00:03<00:00,  2.17it/s]

tuning metric:0.006599
tuning hyper-params {'regressor_beta_prior': [0.1, 0.1, 0.1], 'regressor_sigma_prior': [1, 1, 1]}


100%|██████████| 9/9 [00:04<00:00,  2.16it/s]

tuning metric:0.0066221


In [11]:
tuned_df.head()

,regressor_beta_prior,regressor_sigma_prior,metrics
0,"[0, 0, 0]","[0.3, 0.3, 0.3]",0.006608
1,"[0, 0, 0]","[0.5, 0.5, 0.5]",0.006609
2,"[0, 0, 0]","[1, 1, 1]",0.006744
3,"[0.05, 0.05, 0.05]","[0.3, 0.3, 0.3]",0.006710
4,"[0.05, 0.05, 0.05]","[0.5, 0.5, 0.5]",0.006442


In [12]:
best_params

[{'regressor_beta_prior': [0.1, 0.1, 0.1],
  'regressor_sigma_prior': [0.3, 0.3, 0.3]}]